In [6]:
import ujson as json
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly.plotly as py
import datetime
from __future__ import division

from moztelemetry import get_pings, get_pings_properties, get_one_ping_per_client

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [49]:
# We grab pings starting from 2 days ago until [TimeWindow] days ago.
TimeWindow = 3

# Additional filters.
Filters = {
    'app': 'Firefox',
    
    # We sample 0.5% of pings. For testing, it is beter to use a small number
    # here (like 0.001) to speed up processing time.
    'fraction': 0.01,
    
    # Optionally restrict pings to a single channel.
     'channel': 'nightly',
}

# In case you need them!
VendorIDs = {
    'Intel': '0x8086',
    'NVIDIA': '0x10de',
    'AMD': '0x1002',
    'ATI': '0x1002'
}

In [50]:
###############################
# This section gathers pings. #
###############################

def run_get_pings():
    def fmt_date(d):
        return d.strftime("%Y%m%d")
    t1 = fmt_date(datetime.datetime.now() - datetime.timedelta(TimeWindow + 0)) # go back 16 days
    t2 = fmt_date(datetime.datetime.now() - datetime.timedelta(0)) # go back 2 days
    return get_pings(sc, build_id=(t1, t2), **Filters)

# Get pings for the parameters in the previous step.
raw_pings = run_get_pings()

In [51]:
######################################################################
# This section takes the raw ping list, then formats and filters it. #
######################################################################

# Map the pings into a more readable dictionary-like form. To see
# what these look like, execute "pings.take(1)".
unique_pings = get_pings_properties(raw_pings, [
  "clientId",
  "environment/build/version",
  "environment/system/os/name",
  "environment/system/os/version",
  "environment/system/os/servicePackMajor",
  "environment/system/os/servicePackMinor",
  "environment/system/gfx/adapters",
  "environment/system/gfx/features",
  "environment/system/cpu"
])
unique_pings = get_one_ping_per_client(unique_pings)

In [52]:
# We add two extra steps. The first rewrites the ping to have some
# information more easily accessible (like the primary adapter),
# and the second step removes any pings that don't have adapter
# information.
def rewrite_ping(p):
    adapters = p.get('environment/system/gfx/adapters', None)
    if not adapters:
        return None
    adapter = adapters[0]
            
    p['adapter'] = adapter
    features = p.get('environment/system/gfx/features', None)
    if not features:
        return None
    feature = features
            
    p['feature'] = feature
            
    # Convert the version to a tuple of integers.
    if 'driverVersion' in adapter:
        p['driverVersion'] = [int(n) for n in adapter['driverVersion'].split('.') if n.isdigit()]
    return p

def filter_ping(p):
    return 'adapter' in p
pings = unique_pings.map(rewrite_ping).filter(filter_ping)
pings = pings.cache()

#pings = pings.cache()

In [53]:
# Observe the format of a random ping. This may take some time since it has to
# execute the pipeline.
#pings.take(1)
#pings.take(1)
#new_pings = pings.map(rewrite_ping).filter(filter_ping)
pings.take(1)

[{'adapter': {u'GPUActive': True,
   u'description': u'Intel(R) HD Graphics',
   u'deviceID': u'0x0046',
   u'driver': u'igdumd64 igd10umd64 igdumdx32 igd10umd32',
   u'driverDate': u'4-10-2011',
   u'driverVersion': u'8.15.10.2361',
   u'subsysID': u'14321043',
   u'vendorID': u'0x8086'},
  'clientId': u'79d4cff0-eb10-42c2-8432-d0197b64a38d',
  'driverVersion': [8, 15, 10, 2361],
  'environment/build/version': u'49.0a1',
  'environment/system/cpu': {u'cores': 2,
   u'count': 4,
   u'extensions': [u'hasMMX',
    u'hasSSE',
    u'hasSSE2',
    u'hasSSE3',
    u'hasSSSE3',
    u'hasSSE4_1',
    u'hasSSE4_2'],
   u'family': 6,
   u'l2cacheKB': 256,
   u'l3cacheKB': 3072,
   u'model': 37,
   u'speedMHz': 2466,
   u'stepping': 5,
   u'vendor': u'GenuineIntel'},
  'environment/system/gfx/adapters': [{u'GPUActive': True,
    u'description': u'Intel(R) HD Graphics',
    u'deviceID': u'0x0046',
    u'driver': u'igdumd64 igd10umd64 igdumdx32 igd10umd32',
    u'driverDate': u'4-10-2011',
    u'dr

In [54]:
# Count the total number of sessions in the dataset.
TotalSessions = pings.count()
print('Number of sessions: {0}'.format(TotalSessions))

Number of sessions: 1079


In [55]:
import json
import urllib2

gpu_db = json.load(urllib2.urlopen('https://raw.githubusercontent.com/jrmuizel/gpu-db/master/intel.json'))
devices = {}
for gen in gpu_db['8086'].items():
    for chipset in gen[1].items():
        for dev in chipset[1]:
            #print dev, gen[0]
            devices[int(dev,16)] = gen[0]

In [56]:
# Sample filter #1 - how many people are using Intel devices
# with a driver less than 8.15.10.2622? (bug 1175366).
def sample_filter_1(p):
    if not 'vendorID' in p['adapter']:
        return False
    if not 'driver' in p['adapter']:
        return False

    if not 'compositor' in p['feature']:
        return False
    
    #if p['feature']['compositor'] != 'basic':
    #    return False
    #if devices[int(p['adapter']['deviceID'],16)] in ('gen6'):
    #    return True

    #if devices[int(p['adapter']['deviceID'],16)] in ('gen7'):

     #   return True
    return True

interesting_pings = pings.filter(sample_filter_1)
interesting_pings.count()

1041

In [57]:
def match_count(a, b):
    print('{0} out of {1} sessions matched. ({2:.2f}%)'.format(
        a.count(),
        b.count(),
       ((a.count() / b.count()) * 100)))

In [60]:
ssse3_pings = interesting_pings.filter(lambda p: 'hasSSSE3' in p['environment/system/cpu']['extensions'])
multicore_pings = interesting_pings.filter(lambda p: p['environment/system/cpu']['cores'] > 1)
quadcore_pings = interesting_pings.filter(lambda p: p['environment/system/cpu']['cores'] > 2)
avx_pings = interesting_pings.filter(lambda p: 'hasAVX' in p['environment/system/cpu']['extensions'])
avx2_pings = interesting_pings.filter(lambda p: 'hasAVX2' in p['environment/system/cpu']['extensions'])


match_count(ssse3_pings, interesting_pings)
match_count(multicore_pings, interesting_pings)
match_count(quadcore_pings, interesting_pings)
match_count(avx_pings, interesting_pings)
match_count(avx2_pings, interesting_pings)

937 out of 1041 sessions matched. (90.01%)
1006 out of 1041 sessions matched. (96.64%)
487 out of 1041 sessions matched. (46.78%)
644 out of 1041 sessions matched. (61.86%)
242 out of 1041 sessions matched. (23.25%)


In [12]:

def webgl_fail(p):
    if p['payload/histograms/CANVAS_WEBGL_SUCCESS'][0] > 0:
        return True

     #   return True
    return False
fail_pings = interesting_pings.filter(webgl_fail)

def vga_fail(p):
    if 'vga' in p['adapter']['driver']:
        return True


     #   return True
    return False
vga_pings = fail_pings.filter(vga_fail)

def not_vga_fail(p):
    if 'vga' in p['adapter']['driver']:
        return False
     #   return True
    return True
not_vga_pings = fail_pings.filter(not_vga_fail)


#TDRResults = sample_result_1.map(lambda p: p[DeviceResetReasonKey]).reduce(lambda x, y: x + y)
#help(pd.Series(TDRResults.values, DeviceResetReason))
#print(pd.Series(TDRResults.values[1:8],DeviceResetReason))
#pd.Series(TDRResults.values[1:8],DeviceResetReason).plot.bar()
#reduce(lambda x, y: sample_result_1.take(2))
#sample_result_1.take(1)[0]['payload/histograms/DEVICE_RESET_REASON'])
#fail_pings.take(1)




def map_x_to_count(data, sourceKey):
    def extract(p):
        return (p['adapter'][sourceKey],)
    return data.map(extract).countByKey()    
match_count(interesting_pings, pings)    
match_count(fail_pings, interesting_pings)
match_count(vga_pings, fail_pings)
#map_x_to_count(not_vga_pings, 'driverVersion')
#not_vga_pings.take(5)

35154 out of 431156 sessions matched. (8.15%)
8770 out of 35154 sessions matched. (24.95%)
1051 out of 8770 sessions matched. (11.98%)
